## Which tokens affect classifier constructions the most?

Load the best model from data augmentation experiments and the test data:

In [12]:
#!g1.1
import numpy as np
import pandas as pd
import torchtext.vocab as vocab
import torch
from razdel import sentenize
import json

In [3]:
#!g1.1
from simpletransformers.t5 import T5Model, T5Args

use_cuda = True

model_args = T5Args()
model_args.special_tokens_list = ["clf", "poss", "sg", "<nums>", "<dact>", "1ps", "2ps", "3ps", "indx", "pl"]
model_args.max_seq_length = 96
model_args.train_batch_size = 30
model_args.eval_batch_size = 30
model_args.num_train_epochs = 15
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 30000
model_args.use_multiprocessing = False
model_args.fp16 = False
model_args.save_steps = -1
model_args.save_eval_checkpoints = False
model_args.no_cache = True
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.preprocess_inputs = False
model_args.num_return_sequences = 1
model_args.wandb_project = "MT5 Russian-RSL Translation"

model = T5Model("mt5", "outputs/best_model", args=model_args)  #model15_special_tokens_mix_all.pth

### Load test data

In [6]:
#!g1.1
test_data = pd.read_csv('test_data.csv', encoding='utf-8')

test_data['stem_rus'] = test_data['test_stem_rus'].apply(lambda sent: sent.strip('[]\'').split('\', \''))
test_data['rsl'] = test_data['test_rsl'].apply(lambda sent: sent.strip('[]\'').split('\', \''))
test_data['gram_rus'] = test_data['test_gram_rus'].apply(lambda sent: json.loads(sent.replace('\'', '\"')))

In [7]:
#!g1.1
for index, row in test_data.iterrows():
    if row['rsl'] == ['']:
        test_data.drop(index, inplace=True)

In [22]:
#!g1.1
test_data['input_text'] = test_data['stem_rus'].apply(' '.join)
test_data['target_text'] = test_data['rsl'].apply(' '.join)

In [23]:
#!g1.1
test_data.head()

,test_stem_rus,test_gram_rus,test_rsl,stem_rus,rsl,gram_rus,input_text,target_text
0,"['а', 'когда', 'дед', 'мороз', 'вернуться', 'д...","[['CONJ'], ['CONJ'], ['им', 'мн', 'S', 'муж', ...","['indx', 'дед.мороз', 'clf', 'clf.группа', 'cl...","[а, когда, дед, мороз, вернуться, домой, мален...","[indx, дед.мороз, clf, clf.группа, clf.перемес...","[[CONJ], [CONJ], [им, мн, S, муж, од], [им, мн...",а когда дед мороз вернуться домой маленький де...,indx дед.мороз clf clf.группа clf.переместить ...
1,"['а', 'мачеха', 'лишать', 'я', 'весь', 'радость']","[['CONJ'], ['им', 'S', 'од', 'жен', 'ед'], ['и...","['мама', 'второй', '3ps', 'все', 'радость', 'д...","[а, мачеха, лишать, я, весь, радость]","[мама, второй, 3ps, все, радость, давать, 1ps,...","[[CONJ], [им, S, од, жен, ед], [изъяв, V, пе, ...",а мачеха лишать я весь радость,мама второй 3ps все радость давать 1ps ноль
4,"['зачем', 'я', 'хотеть', 'изменять', 'свой', '...","[['ADVPRO'], ['им', 'ед', '1-л', 'SPRO'], ['из...","['зачем', 'желание', 'изменение', 'все4', 'при...","[зачем, я, хотеть, изменять, свой, привычка, т...","[зачем, желание, изменение, все4, привычка, тр...","[[ADVPRO], [им, ед, 1-л, SPRO], [изъяв, V, пе,...",зачем я хотеть изменять свой привычка тратить ...,зачем желание изменение все4 привычка тратить ...
5,"['оказываться', 'впервые', 'это', 'появляться'...","[['изъяв', 'V', '3-л', 'нп', 'несов', 'ед', 'н...","['америка', 'есть', 'poss', '3ps', '3ps', 'рас...","[оказываться, впервые, это, появляться, в, аме...","[америка, есть, poss, 3ps, 3ps, рассказ, 1ps, ...","[[изъяв, V, 3-л, нп, несов, ед, непрош], [ADV]...",оказываться впервые это появляться в америка,америка есть poss 3ps 3ps рассказ 1ps интересн...
6,"['я', 'вчера', 'не', 'стирать', 'белье']","[['им', 'ед', '1-л', 'SPRO'], ['ADV'], ['PART'...","['1ps', 'вчера', 'стирать', '<dact>', 'не', 'б...","[я, вчера, не, стирать, белье]","[1ps, вчера, стирать, <dact>, не, был]","[[им, ед, 1-л, SPRO], [ADV], [PART], [изъяв, V...",я вчера не стирать белье,1ps вчера стирать <dact> не был


Try predicting something:

In [19]:
#!g1.1
text = """
    Я умею переводить текст на русский жестовый язык.
    Этот текст перевела модель машинного перевода.
    Дети играли в мяч на улице.
    Получается довольно-таки глупо
    """
to_rsl = [_.text for _ in list(sentenize(text))]
to_rsl

['Я умею переводить текст на русский жестовый язык.',
 'Этот текст перевела модель машинного перевода.',
 'Дети играли в мяч на улице.',
 'Получается довольно-таки глупо']

In [20]:
#!g1.1
rsl_preds = model.predict(to_rsl)

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/4 [00:00<?, ?it/s]

In [21]:
#!g1.1
rsl_preds

['я знать переводить текст на русский жестовый язык',
 'перевести текст перевод модель этот машинный перевод',
 'ребенок играть мяч в улица на',
 'получать2 <dact> глупо']

### Preparing data

- selecting sentences that have clf in target
- saving those input texts and target texts in a separate dataset
- droping subjects -> checking number of classifiers
- droping objects -> checking number of classifiers

If I drop verbs, for sure classifiers will disappear, so it does not make any sense. 

Then I need to separately drop figures and grounds, but how do I annotate them? I save my separate dataset as csv, then open it in google docs and introduce two new columns figure ground, where I put respective noun phrases. Then I save it as a csv again

In [ ]:
#!g1.1
for index, row in test_data.iterrows():
    if 'clf' in row['target_text']:
        print(row['target_text'])